In [2]:
import src.helper.sql_helper as sh
import src.models.prompt as pr
import json
import pprint as pp
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
# import ollama

# response = ollama.chat(model='phi3', messages=[
#     {'role': 'system', 'content': 'You are a helpful AI assistant.'},
#     {'role': 'user', 'content': 'Explain the Capacitated Vehicle Routing Problem (CVRP).'}
# ])

# print(response['message']['content'])


## Create SQL Database usin CSV

In [4]:
# db_path = "./databases"  # Change this to your desired directory
# db_name = "Main.db"
# # csv_file_path = r"C:\Users\csalian\OneDrive - Kmart Australia Limited\2025\Idea_Forge\dandadan\data\raw\employee_tracker.csv"
# csv_file_path = r"C:\Users\csalian\OneDrive - Kmart Australia Limited\2025\Idea_Forge\dandadan\data\raw\PRODUCT_LOCATION_DATA.csv"

# table_name = "PRODUCT_LOCATION_DATA"

# # PRODUCT_LOCATION_DATA, "EMPLOYEE_TRACKER"

# db_handler = sh.CSVtoSQLite(db_path, db_name)
# db_handler.create_table_from_csv(csv_file_path, table_name)
# db_handler.close_connection()

In [5]:
# db_name = "./databases/Main.db"  # Use the correct path if saved in a different directory
# table_name = "EMPLOYEE_TRACKER"

# df = sh.read_table_from_db(db_name, table_name)
# df.head()

In [6]:
import ollama
import sqlite3

def get_database_query(user_input, table_related_info):
    # Define the system message to set up the context for the assistant

    system_message = {
        'role': 'system',
        'content': f"""
            You are an Expert AI assistant that helps generate SQL queries based on user input in English.
            Ignore all previous instructions and conversations. Start a fresh session.
            
            The database schema is as follows:
            
            Tables names and their column names along with their datatypes are as follows:
            {table_related_info}
            
            **Instructions**:
            1. Read the question carefully
            2. Use only the provided schema
            3. Return ONLY the SQL query 
            4. Use SQLite syntax
            5. remove ''' from begining and end of the reposnse also dont add 'sql' in the response. 

            Few examples for you to understand the response are as follows:
            User Input:
            1. "List all sales ordered by date in descending order."
            reposnse:
                'SELECT * FROM sales ORDER BY date DESC;'
            2. "Show the total, average, and maximum sales amount for each product."
            reposnse:
                'SELECT product, SUM(amount) AS total_sales, AVG(amount) AS average_sales, MAX(amount) AS max_sales FROM sales GROUP BY product;'

            NOTE: Strictly follow the instructions to get the correct response.
        """
    }

    
    # Define the user message containing the query
    user_message = {'role': 'user', 'content': user_input}
    
    # Use the ollama.chat method to get the query generated by the model
    response = ollama.chat(model="phi3", messages=[system_message, user_message])
    
    # Extract the query and message from the response
    query = response['message']['content']
    print(f"Generated SQL Query: {query}")
    return query


def execute_query(query):
    # Connect to your database (example: SQLite)
    connection = sqlite3.connect('./databases/Main.db')
    cursor = connection.cursor()
    
    
    # Execute the generated query
    print(f"Executing SQL Query: {query}")
    query = sh.extract_sql_query(query)
    cleaned_query = sh.clean_response(query)
    print(f"Cleaned SQL Query: {cleaned_query}")

    cursor.execute(query)
    
    # Fetch results
    results = cursor.fetchall()
    
    # Get column names
    column_names = [description[0] for description in cursor.description]
    

    # Close connection
    connection.close()
    
     # Convert results to DataFrame
    df = pd.DataFrame(results, columns=column_names)
    

    return df



In [27]:
def determine_relevant_table(user_query, table_metadata):
    prompt = f"""
    You are a database assistant. Given the following table metadata:
    {json.dumps(table_metadata, indent=2)}
    
    Determine which table(s) are needed to answer the following user query:
    "{user_query}"
    
    Return only the table names as a JSON list.
    PLEASE NOTE : DONOT MERGE THE TABLES AT ANY COST.
    You MUST strictly return None, If the user query does not logically require data from any of the table.

    The output format is as follows:
    if the tables are needed:
    ['table1', 'table2']
    if the tables are not needed:
    ['None']
    """
    
    response = ollama.chat(model="phi3", messages=[{"role": "user", "content": prompt}])
    
    return response["message"]["content"]


In [36]:
# Example usage
table_info = sh.get_database_structure(db_path='./databases/Main.db')
data = json.loads(table_info)
table_metadata = str(data['tables'])


# user_input = "in which location does the Sneakers product present in S001 store"
user_input = "Are you AI ?"

# database_query = get_database_query(user_input, table_metadata)


In [ ]:
table_to_use = determine_relevant_table(user_input, table_metadata)
table_to_use

In [ ]:
sh.extract_list_from_string(table_to_use)


In [ ]:
if not sh.is_sql_related(database_query):
    flag = False
    print("I can only help with SQL queries.")
elif not sh.check_tables_in_query(response_query=database_query, 
                              database_name= "./databases/Main.db"):
    flag = False
    print("I can only help with SQL queries.")
else:
    # Execute query and fetch results
    flag = True
    results = execute_query(database_query)



In [ ]:
results

In [9]:
table_info = sh.get_database_structure(db_path='./databases/Main.db')
data = json.loads(table_info)
table_related_info = str(data['tables'])

In [ ]:
table_related_info

In [ ]:
pp.pprint(table_related_info)

In [10]:
def execute_query(query):
    # Connect to your database (example: SQLite)
    connection = sqlite3.connect('./databases/Main.db')
    cursor = connection.cursor()
    
    # Print the query to debug
    print(f"Executing SQL Query: {query}")
    
    # Execute the generated query
    cursor.execute(query)
    
    # Fetch results
    results = cursor.fetchall()
    
    # Close connection
    connection.close()
    
    return results

In [ ]:
execute_query(query="SELECT COUNT(DISTINCT store_id) FROM PRODUCT_LOCATION_DATA")